# NetPyNE

In [ ]:
from netpyne import specs, sim   

# Network parameters
netParams = specs.NetParams()  # object of class NetParams to store the network parameters

MorphoName =  'dend-C231296A-P4B2_axon-C200897C-P2_-_Scale_x1.000_y0.975_z1.000.asc'
hocName = 'cADpyr_L4UPC'
cellName = 'L4_UPC'

cellRule = netParams.importCellParams(label=cellName, somaAtOrigin=False,
    conds={'cellType': cellName, 'cellModel': 'HH_full'},
    fileName='cellwrapper.py',
    cellName='loadCell',
    cellInstance = True,
    cellArgs={'hocName': hocName, 'MorphoName': MorphoName})
# netParams.renameCellParamsSec(label=cellName, oldSec='soma_0', newSec='soma')
# del netParams.cellParams[cellName]['secs']['myelin_0']

# netParams.cellParams[cellName]
for cellName in netParams.cellParams.keys():    
    netParams.popParams[cellName] = {'cellType': cellName, 'numCells': 1, 'cellModel': 'HH_full'}

# Options
threshold = 100
ampstim =  [-0.0896244038173676, threshold*(0.34859375/250)]

netParams.stimSourceParams['holding'] = {'type': 'IClamp', 'del': 0.0, 'dur': 3000.0, 'amp': ampstim[0]}
netParams.stimSourceParams['step'] = {'type': 'IClamp', 'del': 700.0, 'dur': 2000.0, 'amp': ampstim[1]}

netParams.stimTargetParams['Input->1'] = {'source': 'holding', 'sec':'soma_0', 'loc': 0.5, 'conds': {'pop':'L4_UPC'}}
netParams.stimTargetParams['Input->2'] = {'source': 'step', 'sec':'soma_0', 'loc': 0.5, 'conds': {'pop':'L4_UPC'}}

#------------------------------------------------------------------------------
#
# SIMULATION CONFIGURATION
#
#------------------------------------------------------------------------------

cfg = specs.SimConfig()					            # object of class SimConfig to store simulation configuration

cfg.coreneuron = False

#------------------------------------------------------------------------------
# Run parameters
#------------------------------------------------------------------------------
cfg.duration = 3.0*1e3 ## Duration of the sim, in ms  
cfg.dt = 0.05
cfg.seeds = {'conn': 4321, 'stim': 1000, 'loc': 4321} 
cfg.hParams = {'celsius': 34, 'v_init': -80.0}  

# Recording 
#--------------------------------------------------------------------------
cfg.recordTraces = {'V_soma': {'sec':'soma_0', 'loc':0.5, 'var':'v'}}  ## Dict with traces to record
cfg.recordStim = False			
cfg.recordTime = True  		
cfg.recordStep = cfg.dt 

#------------------------------------------------------------------------------
# Saving
#------------------------------------------------------------------------------
cfg.simLabel = 'v0_batch0'       #   + str(cfg.cynradNumber)
cfg.saveFolder = 'netpyne_recordings/'+cfg.simLabel
# cfg.filename =                	## Set file output name
cfg.savePickle = False	        	## Save pkl file
cfg.saveJson = False           	## Save json file
cfg.saveDat = True           	## Save 
cfg.saveDataInclude = ['simData'] ##   ['simData', 'simConfig', 'net', 'netParams'] ## ['simData'] ##  ['simData'] ##, , 'simConfig', 'netParams'
cfg.backupCfgFile = None 		##  
cfg.gatherOnlySimData = False	##  
cfg.saveCellSecs = False			
cfg.saveCellConns = False	

#------------------------------------------------------------------------------
# Analysis and plotting 
# ------------------------------------------------------------------------------
cfg.analysis['plotTraces'] = {'include': [0], 'timeRange': [0,cfg.duration], 'saveFig': True, 'showFig': True, 'figSize':(12,4)} # , 'ylim': [-90,30] Plot recorded traces for this list of cells

sim.initialize(
    simConfig = cfg, 	
    netParams = netParams)  				# create network object and set cfg and net params
sim.net.createPops()               			# instantiate network populations
sim.net.createCells()              			# instantiate network cells based on defined populations
sim.net.connectCells()            			# create connections between cells based on params
sim.net.addStims() 							# add network stimulation
sim.setupRecording()              			# setup variables to record for each cell (spikes, V traces, etc)
sim.runSim()                      			# run parallel Neuron simulation  
sim.gatherData();                  			# gather spiking data and cell info from each node
sim.saveData()                    			# save params, cell info and sim output to file (pickle,mat,txt,etc)#
sim.analysis.plotData()         			# plot spike raster etc